In [1]:
import face_recognition
import cv2
import numpy as np
import sys
import dlib
from collections import deque
import os

In [2]:
class FaceTracker:
    def __init__(self, buffer_size=10):
        self.face_locations_buffer = deque(maxlen=buffer_size)
        self.smoothed_locations = None
        self.alpha = 0.3  # Smoothing factor
        self.min_detection_confidence = 0.6
        self.last_known_locations = None
        self.frame_width = None
        self.frame_height = None
        
    def update_frame_size(self, width, height):
        self.frame_width = width
        self.frame_height = height
        
    def smooth_locations(self, current_locations):
        """Apply temporal smoothing to face locations"""
        if not current_locations:
            self.face_locations_buffer.append([])
            return []
            
        self.face_locations_buffer.append(current_locations)
        
        if not self.smoothed_locations:
            self.smoothed_locations = current_locations
            return current_locations
            
        # Match faces across frames
        smoothed = []
        for curr_face in current_locations:
            matched = False
            for prev_face in self.smoothed_locations:
                # Calculate IOU between current and previous face
                iou = self.calculate_iou(curr_face, prev_face)
                if iou > 0.3:  # IOU threshold
                    # Apply exponential smoothing
                    smooth_face = tuple(int(self.alpha * c + (1 - self.alpha) * p)
                                     for c, p in zip(curr_face, prev_face))
                    smoothed.append(smooth_face)
                    matched = True
                    break
            
            if not matched:
                smoothed.append(curr_face)
                
        self.smoothed_locations = smoothed
        return smoothed
        
    def calculate_iou(self, box1, box2):
        """Calculate Intersection over Union between two bounding boxes"""
        top1, right1, bottom1, left1 = box1
        top2, right2, bottom2, left2 = box2
        
        # Calculate intersection
        inter_top = max(top1, top2)
        inter_right = min(right1, right2)
        inter_bottom = min(bottom1, bottom2)
        inter_left = max(left1, left2)
        
        if inter_right < inter_left or inter_bottom < inter_top:
            return 0.0
            
        inter_area = (inter_right - inter_left) * (inter_bottom - inter_top)
        
        # Calculate union
        box1_area = (right1 - left1) * (bottom1 - top1)
        box2_area = (right2 - left2) * (bottom2 - top2)
        union_area = box1_area + box2_area - inter_area
        
        return inter_area / union_area if union_area > 0 else 0

In [3]:
def load_known_face(image_path):
    """Safely load and encode a known face"""
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Could not load image {image_path}")
            return None
            
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_image, model="hog")
        
        if len(face_locations) != 1:
            print(f"Error: Found {len(face_locations)} faces in {image_path}. Expected 1 face.")
            return None
        
        encoding = face_recognition.face_encodings(rgb_image, face_locations)[0]
        return encoding
        
    except Exception as e:
        print(f"Error loading {image_path}: {str(e)}")
        return None

In [4]:
def load_known_faces(base_folder):
    """Load all known faces from the specified folder structure"""
    known_face_encodings = []
    known_face_names = []
    
    # Walk through the base folder
    try:
        for person_folder in os.listdir(base_folder):
            person_path = os.path.join(base_folder, person_folder)
            
            # Skip if not a directory
            if not os.path.isdir(person_path):
                continue
                
            person_name = person_folder
            person_encodings = []
            
            # Process each image in the person's folder
            for image_file in os.listdir(person_path):
                if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(person_path, image_file)
                    try:
                        # Load and encode face
                        image = cv2.imread(image_path)
                        if image is None:
                            print(f"Warning: Could not load image {image_path}")
                            continue
                            
                        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                        face_locations = face_recognition.face_locations(rgb_image, model="hog")
                        
                        if len(face_locations) != 1:
                            print(f"Warning: Found {len(face_locations)} faces in {image_path}. Expected 1 face.")
                            continue
                        
                        encoding = face_recognition.face_encodings(rgb_image, face_locations)[0]
                        known_face_encodings.append(encoding)
                        known_face_names.append(person_name)
                        print(f"Successfully loaded: {image_path}")
                        
                    except Exception as e:
                        print(f"Error processing {image_path}: {str(e)}")
                        continue
    
    except Exception as e:
        print(f"Error accessing the known_faces directory: {str(e)}")
        return [], []
        
    return known_face_encodings, known_face_names

In [5]:
def process_frame(frame, known_face_encodings, known_face_names, face_tracker):
    """Process a single frame with smooth tracking"""
    try:
        # Update frame size in tracker
        height, width = frame.shape[:2]
        face_tracker.update_frame_size(width, height)
        
        # Resize frame for faster processing
        scale_factor = 0.25
        small_frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        
        # Detect faces
        face_locations = face_recognition.face_locations(rgb_small_frame, model="hog")
        face_names = []
        
        if face_locations:
            # Scale up locations
            face_locations = [(int(top/scale_factor), int(right/scale_factor), 
                             int(bottom/scale_factor), int(left/scale_factor)) 
                            for top, right, bottom, left in face_locations]
            
            # Apply smoothing
            face_locations = face_tracker.smooth_locations(face_locations)
            
            # Get encodings for detected faces
            face_encodings = face_recognition.face_encodings(frame, face_locations)
            
            # Recognize faces
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.6)
                name = "Unknown"
                
                if True in matches:
                    face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                    best_match_index = np.argmin(face_distances)
                    if matches[best_match_index]:
                        name = known_face_names[best_match_index]
                        
                face_names.append(name)
                
        return face_locations, face_names
        
    except Exception as e:
        print(f"Error processing frame: {str(e)}")
        return [], []

In [ ]:
def main():
    # Initialize video capture with increased buffer size
    video_capture = cv2.VideoCapture(0)
    video_capture.set(cv2.CAP_PROP_BUFFERSIZE, 3)
    
    if not video_capture.isOpened():
        print("Error: Could not open video file")
        return

    # Load known faces from directory structure
    print("Loading known faces...")
    known_face_encodings, known_face_names = load_known_faces("known_faces")

    if not known_face_encodings:
        print("Error: No valid face encodings could be loaded")
        return

    print(f"Successfully loaded {len(known_face_encodings)} face(s)")

    # Initialize face tracker
    face_tracker = FaceTracker(buffer_size=5)
    frame_count = 0

    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("\nEnd of video stream")
            break

        # Process frame
        face_locations, face_names = process_frame(frame, known_face_encodings, known_face_names, face_tracker)
        
        # Display results
        status_text = f"Detected: {len(face_locations)} face(s)"
        cv2.putText(frame, status_text, (10, 30), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 0), 1)

        # Draw boxes and labels
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Draw box
            color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
            cv2.rectangle(frame, (left, top), (right, bottom), color, 2)

            # Draw label with confidence
            label_y = bottom - 10 if bottom - 10 > 10 else bottom + 10
            cv2.putText(frame, name, (left, label_y),
                       cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), 1)

        print(f"\rFrame {frame_count}: {status_text}", end="", flush=True)
        frame_count += 1

        # Display the frame
        cv2.imshow('Face Recognition', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Clean up
    video_capture.release()
    cv2.destroyAllWindows()
    print("\nProgram ended")

if __name__ == "__main__":
    main()

Loading known faces...
Successfully loaded: known_faces/Abhay/WhatsApp Image 2024-10-22 at 13.55.09.jpeg
Successfully loaded: known_faces/Abhay/person_image.jpg
Successfully loaded: known_faces/Amon Gus/known_person2.jpg
Successfully loaded 3 face(s)
Frame 0: Detected: 1 face(s)

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/shobhits/Documents/Mini Project/Phase 2/Criminal Detection/env/lib/python3.12/site-packages/cv2/qt/plugins"


Frame 162: Detected: 1 face(s)

KeyboardInterrupt: 